In [32]:
from dotenv import load_dotenv
load_dotenv()
import os
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec


In [33]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index(
    name = "rag", dimension = 1536 , metric = "cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1"
    )
)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': '030d2226de8d47ca8c8f1585d2ef5f81', 'Date': 'Fri, 23 Aug 2024 21:38:57 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [34]:
import json
data = json.load(open("reviews.json"))
data['Reviews']


[{'professor': 'Dr. Emily Johnson',
  'subject': 'Computer Science',
  'rating': 5,
  'reviews': 'Excellent professor! Very knowledgeable and approachable.'},
 {'professor': 'Dr. John Smith',
  'subject': 'Mathematics',
  'rating': 4,
  'reviews': 'Great at explaining complex concepts but sometimes a bit fast-paced.'},
 {'professor': 'Dr. Amanda Lee',
  'subject': 'Physics',
  'rating': 3,
  'reviews': 'Good professor, but the course content was quite difficult.'},
 {'professor': 'Dr. Robert Brown',
  'subject': 'Chemistry',
  'rating': 5,
  'reviews': 'Very organized and clear in teaching. Highly recommend!'},
 {'professor': 'Dr. Sarah Miller',
  'subject': 'Biology',
  'rating': 2,
  'reviews': 'Not very engaging, and the exams were tough.'},
 {'professor': 'Dr. David Wilson',
  'subject': 'History',
  'rating': 4,
  'reviews': 'Interesting lectures and fair grading.'},
 {'professor': 'Dr. Linda Garcia',
  'subject': 'Economics',
  'rating': 3,
  'reviews': 'Average experience. Some 

In [35]:
processed_data = []
client = OpenAI()
for review in data['Reviews']:
    response = client.embeddings.create(
        input= review['reviews'],
        model = "text-embedding-3-small",
    )
    embedding = response.data[0].embedding
    processed_data.append({
        "values": embedding,
        "id":review["professor"],
        "metadata":{
            "review":review["reviews"],
            "subject":review["subject"],
            "stars":review["rating"]
        }
    })

In [36]:
processed_data[0]

{'values': [-0.011974469,
  -0.06290911,
  -0.06475036,
  0.030968674,
  -0.009346862,
  -0.0075184065,
  -0.030968674,
  0.041248944,
  -0.043064613,
  -0.01750459,
  0.02355256,
  -0.026314422,
  0.013707027,
  -0.0030559509,
  0.0034171669,
  -0.002482161,
  0.0090719545,
  0.010887625,
  0.035546206,
  0.03902411,
  0.019307472,
  -0.028436966,
  0.008873765,
  0.022504074,
  -0.027132753,
  -0.028948423,
  0.025125287,
  0.01820784,
  0.009864712,
  0.022759803,
  0.09543773,
  -0.007914785,
  -0.03321908,
  -0.025342656,
  -0.044522263,
  0.03774547,
  -0.0063772197,
  0.02920415,
  -0.0133106485,
  0.018847162,
  0.037003856,
  0.009442761,
  0.020509394,
  0.022465715,
  0.008995236,
  -0.019115677,
  -0.036594693,
  0.009519479,
  0.03994473,
  0.02920415,
  -0.02233785,
  -0.020343171,
  0.050404012,
  0.018949453,
  -0.06582441,
  0.023322403,
  0.0090719545,
  0.008995236,
  0.019307472,
  -0.03904968,
  0.047284126,
  -0.0030127969,
  0.017159356,
  -0.023526985,
  -0.0166

In [37]:
index = pc.Index('rag')
index.upsert(
    vectors= processed_data,
    namespace="ns1"
)

{'upserted_count': 30}

In [38]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 30}},
 'total_vector_count': 30}